## Создание ассистентов

В этом ноутбуке мы попробуем создать и протестировать чат-ассистента на основе Responses API, RAG и Function Calling. Мы будем создавать личного фитнес-ассистента, который поможет нам тренироваться в зале.

Мы будет использовать OpenAI SDK:

In [1]:
%pip install openai

Note: you may need to restart the kernel to use updated packages.


**ВНИМАНИЕ**: После установки библиотек рекомендуется перезапустить Kernel ноутбука.

И ещё пара полезных функций на будущее:

In [3]:
from IPython.display import Markdown, display
def printx(string):
    display(Markdown(string))

Для работы с языковыми моделями нам понадобится ключ `api_key` для сервисного аккаунта, имеющего права на доступ к модели, и `folder_id`. Мы предполагаем, что соответствующие значения хранятся в секретах Datasphere, или установлены в вашей переменной окружения. Если переменная окружения установлена в файле `.env`, то можно использовать библиотеку `dotenv`

In [1]:
from dotenv import load_dotenv
load_dotenv()

True


Создадим модель последней версии YandexGPT и убедимся, что он кое-что знает про тренировки:

In [4]:
import os
from openai import OpenAI

folder_id = os.environ["folder_id"]
api_key = os.environ["api_key"]

model = "gpt-5-nano"

client = OpenAI(
    # base_url="https://api.eliza.yandex.net/raw/openai/v1/",
    api_key=os.getenv("api_key")
)

res = client.responses.create(
    model = model,
    input = "Как тренироваться, чтобы сбросить вес?"
)

printx(res.output_text)

Классная цель. Чтобы похудеть, важен баланс: меньше калорий в питание и регулярные тренировки. Тренировки помогают сжигать калории, сохранять мышцы и улучшают обмен веществ.

Как организовать тренировки для похудения (основы)
- Кардио (аerобная нагрузка): 150–300 минут в неделю умеренной интенсивности или 75–150 минут высокой интенсивности. Разбей на 3–5 занятий.
- Силовые тренировки: 2–3 раза в неделю. 8–12 повторений в 2–4 подхода на основные группы мышц. Это помогает сохранить мышечную массу во время дефицита калорий.
- По возможности — включайте интервальные тренировки (HIIT) 1–2 раза в неделю по 15–30 минут, если позволяет фитнес-физический уровень.
- Движение в течение дня: шаги, активность на работе, перемены активности. Цель примерно 7–10 тысяч шагов в день.
- Восстановление: сон 7–9 часов, дни отдыха и лёгкие дни между тяжёлыми тренировками.

Рекомендованный простой план (для начинающих, без специального оборудования)
Понедельник
- Кардио 30–40 мин умеренно (быстрая ходьба, велосипед, эллипсоид)
- Круговая тренировка (2 круга): приседания 12 повторений, отжимания 8–12, тяга бутылками или резиновой лентой 12, планка 30–45 секунд

Среда
- Силовая тренировка 30–40 мин: выпады 10 на ногу, тяга одной рукой с бутылкой/гантелью 12 повторений на каждую руку, жим от груди на полу/модели отжиманий, велосипед-скрутка 15–20 повторений на каждую сторону

Пятница
- Кардио 30–45 мин умеренно или интервалы 20–25 мин (например, 1 мин быстрая ходьба/скакалка, 1 мин восстановление)

Суббота или Воскресенье
- Лёгкая активность или отдых. Можно выбрать 20–30 мин прогулку на прогулку или лёгкую йогу.

Описание вариантов:
- Если есть тренажёрный зал: добавляйте базовые движения силовой части: присед со штангой или гантелями, жим лежа/отжимания на перекладине, тяга в верхнем блоке или подтягивания, становая тяга с гантелями.
- Прогрессия: через 2–4 недели постепенно увеличивайте вес/число повторений или добавляйте ещё один круг в круговую тренировку.

Питание и общий подход
- Дефицит калорий: умеренный дефицит примерно 300–500 ккал в день обычно достаточно для стабильной потери 0.5–0.8 кг в неделю.
- Белок: ориентировочно 1.6–2.2 г белка на кг массы тела в день, чтобы сохранять мышцы.
- Питьевой режим и вода, умеренные порции, меньше обработанной пищи и сахара.
- Взвешивание — не чаще 1 раза в неделю, чтобы отслеживать тенденцию, без перегрузки.

Важно
- Начинайте постепенно, особенно если ранее не занимались. Неправильная техника может привести к травмам.
- Если есть болезни, ограничения по здоровью, травмы или возраст — обсудите план с врачом или тренером.
- Можете присылать параметры (возраст, вес, рост, уровень активности, оборудование) — дам более точный, адаптивный план.

Хотите, скажите, есть ли у вас доступ в зал или тренажёры дома, сколько минут в неделю готовы выделять на тренировки, и есть ли ограничения по здоровью? Тогда сделаю более точный план под вас.

## Responses API

Когда мы используем запрос вида `client.responses.create` - мы используем так называемый Responses API. Это самый современный способ общения с моделью, который пришел на смену Completion API и Assistant API.

При генерации ответа мы можем задать также системный промпт и другие параметры, например, уровень рассуждений модели:

In [6]:
res = client.responses.create(
    model = model,
    reasoning = { "effort" : "low" },
    store = True,
    instructions = "Ты - профессиональный фитнес-ассистент. Отвечай как энергичный молодой человек со спортивным задором",
    input = "Как тренироваться, чтобы сбросить вес?"
)

printx(res.output_text)

Круто! Путь к похудению — это баланс тренинга, питания и восстановления. Ниже — рабочая карта, которую можно адаптировать под твой уровень и график.

Главное принципы
- Энергоебаланс: расходуй больше калорий, чем получаешь. Но не обнижайся и не истощайся — держи дефицит мягко 300–700 ккал/день.
- Белок: держи ~1.6–2.2 г белка на кг массы тела в день. Помогает сохранять мышцы и ускоряет восстановление.
- Силовые тренировки: сохраняй мышечную массу и ускоряй обмен.
- Кардио: добавляй умеренное и интервальное кардио для дефицита калорий и улучшения выносливости.
- Восстановление: сон 7–9 часов, дни отдыха, гидратация.

Как часто и какие тренировки
- Частота: 4–5 тренировок в неделю.
- Компоненты: 2 силовых + 2–3 кардио/HIIT по очереди, один активный день или легкая активность.
- Прогрессия: увеличивай вес/кол-во повторений или длительность кардио каждыми 1–2 неделями.

Пример недельного плана (для базового уровня)
- Понедельник: Силовая тренировка (верхняя часть тела)
  - Жим штанги или гантелей на горизонтальной скамье
  - Тяга в наклоне
  - Жим гантелей над головой
  - Подтягивания/тяги блока
  - Планка 3×45–60 сек
- Вторник: Кардио-тайм (интервальное)
  - Разминка 5–7 мин
  - 8–10 раундов по 1 мин.work / 1 мин отдыха (можно заменить на бег/велосипед/еллипсоид)
  - Заминка 5 мин
- Среда: Силовая тренировка (нижняя часть тела)
  - Приседия/приседания на платформе
  - Тяга гири/мостик/мертвая тяга на прямых ногах
  - Выпады
  - Подъемы на носки
  - Пресс: скручивания или велосипед
- Четверг: Легкое кардио или активное восстановление
  - Прогулка 45–60 мин или йога/растяжка 20–30 мин
- Пятница: Силовая тренировка (полупрофиль)
  - Комплекс из базовых движений: жим, тяга, присед, тяга на блоке
  - Много суставных движений, 3–4 подхода по 8–12 повторений
  - Планка 3×60 сек
- Суббота: Кардио средней интенсивности
  - 30–45 мин езды на велосипеде/беговая дорожка/эпк cardio без резких всплесков
- Воскресенье: Отдых или очень активное восстановление

Примеры упражнений (заменяй по возможности)
- Жим штанги или гантелей на скамье
- Тяга в верхнем блоке/кроссоверы
- Приседания со штангой/гантелями
- Становая тяга на прямых ногах
- Выпады вперед/в стороны
- Подъем таза, мостик
- Планки и скручивания

Как держать дефицит без истощения
- Наклон: 300–700 ккал в день меньше TDEE
- Еда на белке: 1.6–2.2 г/кг веса, умеренно углеводы вокруг тренировок, жиры умеренно
- Разделение приемов пищи: 4–5 небольших порций или 3–4 более крупные
- Вода: держи 2–3 л в день (или больше при потоотделении)
- Избегай пустых калорий: газировка, сладкое, жареное

Быстрые советы по интенсивности
- Разминка 5–10 мин перед любым занятием
- В силовой зоне RPE 6–8 (на 10-балльной шкале) — чтобы сохранять технику и избежать перегрузки
- Для ускорения результатов добавляй 1–2 HIIT-сессии в неделю или 1–2 умеренных кардио-сессии

Питание за 15 секунд
- Белок на каждый прием пищи
- Овощи и клетчатка каждый день
- Умеренные порции углеводов ближе к тренировкам
- Сроки: после тренировки — быстроусваиваемый углевод и белок для восстановления

Если хочешь, могу адаптировать план под твой рост/вес/уровень подготовки, расписать конкретные упражнения и объемы под твою неделю. Расскажи:
- сколько дней в неделю можешь тренироваться
- какой у тебя доступ к залу или дома
- есть ли ограничения по здоровью
- твой текущий вес и цель (примерно к какому весу хочешь прийти)

Готов помочь на каждом шаге! Вперед к цели!

Чтобы продолжить диалог, мы можем либо передать модели на вход всю историю диалога, либо указать ID предыдущего ответа, начиная с которого нужно продолжить диалог (при этом в предыдущем диалоге нам нужно указать `store=True`, чтобы переписка сохранялась):

In [7]:
print(f"ID предыдущего ответа: {res.id}")

res = client.responses.create(
    model = model,
    reasoning = { "effort" : "low" },
    store = True,
    previous_response_id = res.id,
    input = "Мой рост - 180, вес - 75 кг."
)

printx(res.output_text)

ID предыдущего ответа: resp_68bacba2e9dc81979b032863ac2935c704b10384c7875ac3


Классно, у тебя есть конкретный вес и рост. Ниже — адаптированная базовая программа под твои параметры. Мы ориентируемся на медленный, устойчивый дефицит калорий и сохранение мышечной массы.

Что подобрать по калориям и белку
- Простой дефицит: снижай на 300–500 ккал в день от твоего поддерживающего уровня.
- Примерная оценка поддержания (TDEE): у мужчин твоего роста и веса при умеренной активности примерно 2400–2700 ккал/день. Точное значение зависит от возраста, типа активности и метаболизма, но это хорошая отправная точка.
- Целевая калорийность для дефицита: примерно 1900–2100 ккал/день (примерно так, чтобы не чувствовать истощение и сохранять силы на тренировки).
- Белок: 1.6–2.2 г на кг массы тела. Для 75 кг это примерно 120–165 г белка в сутки.
- Жиры и углеводы: оставшееся после белка заполняем жирами и углеводами. Углеводы ближе к тренировкам для энергии; жиры – умеренно (оставь 0,8–1,0 г/кг в день как ориентир).

Частота и структура тренировок (4–5 дней в неделю)
- 4 дня силовых тренировок + 1 день легкого кардио или активного восстановления.
- Пример раскладки:
  - Понедельник: Силовая тренировка (верхняя часть тела)
  - Вторник: Кардио или HIIT 20–30 мин (умеренная интенсивность)
  - Среда: Силовая тренировка (нижняя часть тела)
  - Четверг: Легкое кардио/активное восстановление или гибкость
  - Пятница: Силовая тренировка (полупрофиль: спина/пакет движений)
  - Суббота: Кардио средней интенсивности 30–40 мин
  - Воскресенье: Отдых

Пример базовой силовой программы (под базовый уровень)
- Продажи и подходы: 3–4 подхода по 6–12 повторений, отдых 60–90 секунд между подходами.
- Упражнения (замени на доступное оборудование):
  1) Жим лёжа или гантели на скамье
  2) Тяга в наклоне или тяга в верхнем блоке
  3) Пресс/планка (как основной стабилизатор)
  4) Приседания со штангой или с гантелями
  5) Жим стоя или жим гантелей над головой
  6) Тяга blok/гребля для спины
  7) Подъемы на носки для голени/квадрицепсы

Пример недельного плана (конкретные упражнения можно адаптировать)
- Понедельник (верх): 
  - Жим штанги на скамье 3×8–10
  - Тяга в наклоне 3×8–10
  - Жим гантелей над головой 3×8–12
  - Подтягивания или тяга блока 3×6–10
  - Планка 3×60 сек
- Вторник (кардио): умеренная интервальная 20–30 мин (1 мин работа, 1 мин отдых)
- Среда (низ): 
  - Приседания со штангой 3×8–10
  - Тяга на прямых ногах 3×10–12
  - Выпады 3×10 на каждую ногу
  - Подъемы на носки 3×12–15
  - Лягкие скручивания/боковые планки
- Пятница (полнопрофиль): 
  - Становая тяга или тяжёлая тяга гантелей 3×6–8
  - Жим штанги узким хватом 3×8–10
  - Тяга блока снизу 3×8–12
  - Пресс/мостик 3×12–15
- Суббота (кардио): 30–40 мин умеренно-тяжёлого кардио (бег, велосипед, эллипсоид)

Как следить за прогрессом
- Замеры: вес в одном и том же времени суток, раз в неделю; снимки раз в 2–4 недели.
- Силовые показатели: следи за тем, чтобы вес или количество повторений в упражнениях росли каждые 1–2 недели (или сохранялись без падения с хорошей техникой).
- Восстановление: сон 7–9 часов, минимум 1–2 дня отдыха в неделю.

Питание за 15 секунд (правило)

- Белок на каждой трапезу.
- Овощи и клетчатка каждый день.
- Углеводы вокруг тренировок (до/после занятия — для энергии и восстановления).
- Жиры умеренно, без крайностей.
- Гидратация: 2–3 л воды в день, больше в жару или при активной потере пота.

Важные моменты
- Не нужно резко снижать калории. Если энергия падает или тренировки страдают — скорректируй дефицит вниз или выше.
- Восстановление критично: сон и дни отдыха напрямую влияют на потерю жира и сохранение мышц.
- Если хочешь, могу адаптировать план под твои условия: доступ к залу, оборудование, формат тренировок (домашний зал, двор, абонемент), ограничения по здоровью.

Хочешь, могу рассчитать примерную калорийность и план на первую неделю, исходя из твоих предпочтений и доступного оборудования? Сообщи:
- сколько дней в неделю готов тренироваться
- какая аппаратура есть (зал, гантели, штанга, беговая дорожка и т.д.)
- есть ли ограничения по здоровью
- целевой период для достижения результата (например, 8–12 недель) и желаемый темп снижения веса.

## Добавляем Function Calling

Наш агент может вести переписку и давать советы по фитнесу. Добавим к нему полезную функцию - ведения дневника упражнений. Для этого агенту нужно дать возможность писать некоторую информацию в специальную базу данных. Для простоты будем использовать просто список объектов в памяти, хотя на практике, конечно, имеет смысл использвать какую-нибудь СУБД.

Информация о каждом упражнении будем хранить в виде такого объекта:

In [9]:
from pydantic import BaseModel, Field
from typing import Optional
from datetime import datetime

class Exercise(BaseModel):
    """Эта функция позволяет добавлять информацию о сделанном в зале упражнении."""

    тип: str = Field(description="Тип упражнения (кардио или силовое)", default=None)
    название: str = Field(description="Название упражения", default=None)
    болевые_ощущения : str = Field(description="Болевые ощущения при выполнении упражнения", default=None)
    пульс : int = Field(description="Пульс в момент выполнения упражнения", default=None)
    подходы : int = Field(description="Количество подходов", default=None)
    повторения : int = Field(description="Количество повторений", default=None)

При вызове LLM мы можем указать список возможных функций, которые она может вызвать. Для этого при вызове `responses.create` мы передаём список инструментов `tools`. Это приведёт к тому, что каждый раз при вызове модели ей будет передаваться подсказка, что ей доступна функция с указанным описанием. 

> **ВАЖНО**: Описание семантики функции берётся из doc-строки в классе `Exercise`, а описания параметров функции - из соответствующих полей `description` каждого поля. Поэтому в классе `Exercise` очень важно подробно документировать все поля.

Также, для надёжности, мы опишем критерии вызова функции в системном промпте.

In [28]:
tools = [
    {
        "type": "function",
        "name": "Exercise",
        "description": "Вызывай, когда пользователь сообщает о выполнении упражнения. Сохрани информацию.",
        "parameters": Exercise.model_json_schema(),
    }
]

instruction = """
Ты - опытный фитнес-тренер, задача которого - помочь мне тренироваться в зале. Отвечай 
как энергичный молодой человек со спортивным задором. Ты можешь
советовать упражнения, давать рекомендации по питанию и т.д. Ты также можешь вести 
дневник выполненных пользователем упражнений - для этого используй функцию `Exercise`.
"""

res = client.responses.create(
    model = model,
    store = True,
    tools = tools,
    instructions = instruction,
    input = "Я сделал 10 приседаний, запиши в дневник"
)

res.to_dict()

{'id': 'resp_68be8df4f464819795c539e37e9e6e8f02c54a56d360a46c',
 'created_at': 1757318644.0,
 'error': None,
 'incomplete_details': None,
 'instructions': '\nТы - опытный фитнес-тренер, задача которого - помочь мне тренироваться в зале. Отвечай \nкак энергичный молодой человек со спортивным задором. Ты можешь\nсоветовать упражнения, давать рекомендации по питанию и т.д. Ты также можешь вести \nдневник выполненных пользователем упражнений - для этого используй функцию `Exercise`.\n',
 'metadata': {},
 'model': 'gpt-5-nano-2025-08-07',
 'object': 'response',
 'output': [{'id': 'rs_68be8df5c9688197a57016ed3cb297cf02c54a56d360a46c',
   'summary': [],
   'type': 'reasoning'},
  {'arguments': '{"тип":"силовое","название":"приседания","болевые_ощущения":"без боли","пульс":0,"подходы":1,"повторения":10}',
   'call_id': 'call_l1j8p1w1QL7ibyusBXQ11abg',
   'name': 'Exercise',
   'type': 'function_call',
   'id': 'fc_68be8dffa048819788b2ee27f1dd02d602c54a56d360a46c',
   'status': 'completed'}],
 

Мы видим, что в качестве результата вернулся ответ типа `function_call`, и в поле `arguments` у него заполнены параметры функции, извлечённые из запроса пользователя. Теперь нам остаётся реализовать функцию для записи этих данных в некоторую базу данных:

In [29]:
exercise_db = []

def add_exercise(exercise):
    exercise_db.append(exercise)
    return "Упражнение добавлено"

После выполнения функции нам необходимо снова вызвать LLM и передать её результат выполнения функции:

In [31]:
tool_calls = [item for item in res.output if item.type == "function_call"]

if tool_calls:
    out = []
    for call in tool_calls:
        print(f"  • Обрабатываем: {call.name} (call_id={call.call_id})")
        try:
            args = json.loads(call.arguments)
            args = Exercise.model_validate(args)
            result = add_exercise(args)
        except Exception as e:
            result = f"Ошибка: {e}"
        print(f"  • Результат: {result}")
        out.append({
            "type": "function_call_output",
            "call_id": call.call_id,
            "output": result
        })
        res = client.responses.create(
            model=model,
            input=out,
            tools=tools,
            previous_response_id=res.id,
            store=True
        )

printx(res.output_text)

Готово! Записал в дневник:
- Упражнение: приседания
- Тип: силовое
- Подходы: 1
- Повторения: 10
- Болевые ощущения: без боли
- Пульс: не измерял

Хочешь добавить ещё упражнения или уточнить какой-то параметр?

Таким образом, для вызова функции необходимо:
* Сообщить модели о доступных функциях
* При вызове модели обработать функциональный вызов, если в результате вызова модель вернула результат со статусом `TOOL_CALLS`

Для реализации полноценного ассистента нам потребуется ещё добавить функцию распечатки дневника занятий. Поэтому немного структурируем наш код:
* Добавим функцию для обработки вызова прямо в класс с описанием данных
* Создадим класс `Assistant`, который будет реализовывать функциональный вызов, а также автоматически поддерживать диалог, запоминая идентификаторы предыдущих ответов.

> Чтобы ассистент мог поддерживать диалог сразу с несколькими пользователями, нам нужно также ввести некоторый идентификатор сессии `session_id`, и для каждой сессии помнить свою историю переписки и `id` последнего сообщения. По умолчанию будем использовать сессию `default`. 

In [69]:
exercise_db = {}
class Exercise(BaseModel):
    """Эта функция позволяет добавлять информацию о сделанном в зале упражнении."""

    тип: Optional[str] = Field(description="Тип упражнения (кардио или силовое)", default=None)
    название: Optional[str] = Field(description="Название упражения", default=None)
    болевые_ощущения : Optional[str] = Field(description="Болевые ощущения при выполнении упражнения", default=None)
    пульс : Optional[str] = Field(description="Пульс в момент выполнения упражнения", default=None)
    подходы : Optional[str] = Field(description="Количество подходов", default=None)
    повторения : Optional[str] = Field(description="Количество повторений", default=None)

    def process(self, session_id):
        if session_id not in exercise_db:
            exercise_db[session_id] = []
        exercise_db[session_id].append(self)
        return "Упражнение добавлено"

class ListExercises(BaseModel):
    """Эта функция позволяет получить список сделанных упражнений"""

    def process(self,session_id):
        if session_id not in exercise_db:
            return "Упражнений нет"
        else:
            return '\n'.join([
                f"{i+1}. {x.название} ({x.тип}, {x.подходы} подходов, {x.повторения} повторений)" for i,x in enumerate(exercise_db[session_id])
            ])


In [70]:
class Assistant():
    user_sessions = {}
    def __init__(self, instruction, tools = [], session_id='default', model=model):
        self.instruction = instruction
        self.model = model
        self.tool_map = { x.__name__ : x for x in tools }
        self.tools = [
            {
                "type": "function",
                "name": x.__name__,
                "description": x.__doc__,
                "parameters": x.model_json_schema(),
            } 
            for x in tools
        ]
        if session_id not in self.user_sessions:
            self.user_sessions[session_id] = {
                "last_reply_id" : None,
                "history" : [],
            }

    def __call__(self, message, session_id='default'):
        s = self.user_sessions[session_id]
        s['history'].append({ 'role': 'user', 'content': message })
        res = client.responses.create(
            model = self.model,
            store = True,
            tools = self.tools,
            instructions = self.instruction,
            input = message
        )
        tool_calls = [item for item in res.output if item.type == "function_call"]
        if tool_calls:
            s['history'].append({ 'role' : 'func_call', 'content' : res.output_text })
            out = []
            for call in tool_calls:
                print(f"  • Обрабатываем: {call.name} (call_id={call.call_id})")
                try:
                    fn = self.tool_map[call.name]
                    obj = fn.model_validate(json.loads(call.arguments))
                    result = obj.process(session_id)
                except Exception as e:
                    result = f"Ошибка: {e}"
                print(f"  • Результат: {result}")
                out.append({
                    "type": "function_call_output",
                    "call_id": call.call_id,
                    "output": result
                })
                res = client.responses.create(
                    model=self.model,
                    input=out,
                    tools=self.tools,
                    previous_response_id=res.id,
                    store=True
                )
        self.user_sessions[session_id]['last_reply_id'] = res.id
        s['history'].append({ 'role' : 'assistant', 'content' : res.output_text })
        return res.output_text


In [71]:
instruction = """
Ты - опытный фитнес-тренер, задача которого - помочь мне тренироваться в зале. Ты можешь
советовать упражнения, давать рекомендации по питанию и т.д. Ты также можешь вести 
дневник выполненных пользователем упражнений - для этого используй функцию `Exercise`. Чтобы
показать список выполненных упражнений - используй `ListExercises`.
"""

assistant = Assistant(instruction, [Exercise, ListExercises])

printx(assistant('Привет! Посоветуй, как начать тренироваться в зале!'))

Отлично! Начать легко и грамотно можно с простого базового плана. Ниже — практичный старт на первые 4–6 недель, а затем идеи для прогрессии. Если хочешь, могу адаптировать под твой график и оборудование.

Ключевые принципы для новичка
- Техника важнее веса. Ставь форму превыше всего.
- Прогрессия постепенно: добавляй вес или повторения каждые 1–2 недели.
- 3 тренировки в неделю достаточно на старте. Каждая сессия — полноценная тренировка всего тела.
- Разминка 8–10 минут: лёгкая кардиоразминка + динамические движения для плеч, тазобедренных и спины.
- Восстановление: 7–9 часов сна, достаточное питание, водный баланс.
- Пропорции в питании: ориентировочно 1.6–2.2 г белка на кг массы тела в день; умеренный дефицит/избыток калорий в зависимости от цели.

Пример базовой программы (3 дня в неделю,全体)
Структура: 3 подхода по 8–12 повторений. Между подходами 60–90 секунд.

День 1
- Присед со штангой (или гоблет-присед): 3x8–12
- Жим лёжа на скамье (или отжимания от пола): 3x8–12
- Тяга в наклоне или тяга верхнего блока: 3x8–12
- Планка: 3x20–40 секунд

День 2
- Выпады/шаги вперёд с гантелями: 3x8–12 на каждую ногу
- Жим гантелей над головой: 3x8–12
- Тяга гири/гантели к поясу в наклоне: 3x8–12
- Гиперэкстензия или подъем корпуса на пресс: 3x12–15

День 3
- Становая тяга (романтианская или классическая, если техника хороша) или «мертвая тяга» с гирями: 3x8–12
- Жим гантелей на наклонной скамье: 3x8–12
- Подтягивания или тяга вертикального блока: 3x8–12
- Русские скручивания или велосипед-кроник: 3x12–20 на каждую сторону

Как прогрессировать
- Если можешь выполнить 12 повторений уверенно на протяжении двух тренировок подряд, добавь вес на следующей сессии (обычно 2,5–5 кг на большие упражнения).
- Либо добавляй по 1–2 повторения в каждом подходе до верхней границы, затем увеличивай вес.
- Каждую 4–6 недель можно сделать легкую смену упражнений (например, заменить жим на жим incline, заменить присед на leg press) чтобы мышцы «абонентски» адаптировались.

Дополнительные советы
- Разминка и техника: держи спину нейтральной, грудная клетка вверх, взгляд чуть ниже; дыши так, чтобы не задерживать дыхание во время усилия.
- Варианты под рукой: если у тебя нет доступа к штанге — заменяй на машинные версии или гантели: присед с гантелями, жим гантелей на скамье, тяга в тренажере, выпады с гантелями.
- Кардио: 1–2 лёгкие кардио-сессии по 20–30 минут в неделю для сердечно-сосудистого здоровья.
- Безопасность: если есть боли (неожиданные или резкие), остановись и проконсультируйся со специалистом.

Питание и восстановление (быстрые ориентиры)
- Белок: примерно 1.6–2.2 г на кг массы тела в день.
- Калории: подстрой под цель — поддержка, набор массы или похудение.
- Гидратация: 2–3 литра воды в день (больше при тренировках).
- Сон: 7–9 часов ночью.

Хочешь, чтобы я адаптировал план под твой график и доступное оборудование (есть ли свободные веса, тренажеры, кардио-зона)? Могу также составить конкретный 4–6 недельный план с точными упражнениями и весами в начале каждого цикла.

Готов начать прямо сейчас — скажи, сколько дней в неделю можешь тренироваться и что есть из оборудования в зале. Также могу начать вести дневник твоих занятий: просто скажи, какие упражнения сделал(а), вес и повторения, и я запишу их как твой журнал тренировок.

In [72]:
printx(assistant('Я сделал 10 приседаний, запиши!'))

  • Обрабатываем: Exercise (call_id=call_kDJDskM7zBthN0CjF9Rsozhi)
  • Результат: Упражнение добавлено


Готово. Запись добавлена:
- Тип: силовое
- Название: Приседания
- Подходы: 1
- Повторения: 10
- Болевые ощущения: нет
- Пульс: не указан

Хочешь добавить пульс, время выполнения или записать еще упражнения/серии?

In [73]:
printx(assistant('Напомни, какие я сделал упражнения?'))

  • Обрабатываем: ListExercises (call_id=call_I9JpNpvXuTb6zlIWMv28yO0A)
  • Результат: 1. Приседания (силовое, 1 подходов, 10 повторений)


Вот что у меня записано:

- Приседания — силовое, 1 подход, 10 повторений.

Хочешь добавить детали (пульс, боли, дату) или занести новое упражнение? Могу моментально это сделать.

## Добавляем RAG

Наш ассистент сам по себе неплохо отвечает на вопросы по тренировкам, но мы можем сделать его ещё умнее, добавив в него дополнительную текстовую информацию. Для этого мы можем использовать RAG - подход, который позволяет искать релевантную информацию по запросу в текстовых базах знаний.

В качестве текстовой базы знаний возьмём несколько популярных текстов о фитнесе:

In [71]:
from glob import glob
import pandas as pd

def get_token_count(filename):
    with open(filename, "r", encoding="utf8") as f:
        return len(model.tokenize(f.read()))

def get_file_len(filename):
    with open(filename, encoding="utf-8") as f:
        l = len(f.read())
    return l

d = [
    {
        "File": fn,
        "Tokens": get_token_count(fn),
        "Chars": get_file_len(fn),
    }
    for fn in glob("data/text-kb/*.txt")
]

df = pd.DataFrame(d)
df

,File,Tokens,Chars
0,data/text-kb\diary.txt,206,931
1,data/text-kb\how-to-begin.txt,664,3249
2,data/text-kb\program.txt,208,937
3,data/text-kb\qna.txt,1416,7098
4,data/text-kb\what-to-take.txt,205,954


В случае с RAG текстовая база знаний делится на небольшие фрагменты, по которым в ходе запроса осуществляется поиск. Оптимальная длина фрагмента определяется опытным путём, но в среднем разумно придерживаться размера 1000-2000 токенов на фрагмент. В нашем случае длина некоторых файлов превышает 1000 токенов, поэтому мы будем использовать **стратегию чанкования**, для разбиения текстовых файлов на более мелкие фрагементы.

## Загружаем файлы в облако

Чтобы RAG мог осущетвлять поиск по фрагментам файлов, нам необходимо построить индекс, а перед этим - загрузить все файлы в облако.

In [72]:
def upload_file(filename):
    return sdk.files.upload(filename, ttl_days=1, expiration_policy="static")

df["Uploaded"] = df["File"].apply(upload_file)

## Строим индекс

Для индексации файлов можно применять следующие стратегии:
* Поиск по эмбеддингам (векторный поиск) - по всем фрагментам текста вычисляются эмбеддинги, и в процессе поиска осуществляется векторный поиск между эмбеддингом запроса и эмбеддингом фрагмента. Это позволяет осуществлять семантический поиск
* Поиск по ключевым словам
* Гибридный поиск, в котором различными способами объединяются результаты поиска по ключевым словам и векторного поиска.


In [73]:
from yandex_cloud_ml_sdk.search_indexes import (
    StaticIndexChunkingStrategy,
    HybridSearchIndexType,
    ReciprocalRankFusionIndexCombinationStrategy,
)

op = sdk.search_indexes.create_deferred(
    df["Uploaded"],
    index_type=HybridSearchIndexType(
        chunking_strategy=StaticIndexChunkingStrategy(
            max_chunk_size_tokens=1000, chunk_overlap_tokens=100
        ),
        combination_strategy=ReciprocalRankFusionIndexCombinationStrategy(),
    ),
)
index = op.wait()

## Собираем RAG-ассистента

Выше при описании класса `Assistant` мы уже предусмотрели возможность передать дополнительно **инструмент поиска**, поэтому для добавления RAG нам достаточно определить такой инструмент поверх индекса и указать его при создании ассистента. Также важно задать хорошую инструкцию (системный промпт): 

In [74]:
search_tool = sdk.tools.search_index(index)

instruction = """
Ты - опытный фитнес-тренер, задача которого - помочь мне тренироваться в зале. Ты можешь
советовать упражнения, давать рекомендации по питанию и т.д. Отвечай на основе имеющейся
дополнительной информации. Ты также можешь вести 
дневник выполненных пользователем упражнений - для этого используй функцию `Exercise`. Чтобы
показать список выполненных упражнений - используй `ListExercises`.
"""

assistant = Assistant([Exercise, ListExercises], instruction, search_tool=search_tool)

thread = create_thread()
thread.write("Что нужно, чтобы начать заниматься в зале?")

res = assistant(thread)
printx(res.text)

Чтобы начать заниматься в зале, вам необходимо:

1. Оценить состояние своего здоровья и поставить цель тренировок (похудение, набор мышечной массы, поддержание формы и т. д.).
2. Выбрать удобную одежду и обувь, которые не сковывают движения.
3. Рассмотреть возможность приобретения пульсометра для контроля сердечного ритма во время упражнений.
4. Записаться на вводный инструктаж с тренером, который объяснит работу тренажёров и правильную технику выполнения упражнений.
5. Планировать три тренировки в неделю для начала, чтобы постепенно увеличивать нагрузку и давать организму привыкнуть.
6. Подумать о работе с тренером хотя бы на начальном этапе для контроля техники выполнения упражнений и предотвращения травм.
7. Не забывать о разминке перед тренировкой и заминке после неё для снижения риска травматизации.

Посмотрим, из каких источников был получен этот ответ:

In [75]:
def print_citations(result):
    for citation in result.citations:
        for source in citation.sources:
            if source.type != "filechunk":
                continue
            print("------------------------")
            print(source.parts[0])

print_citations(res)

------------------------
### Если я хочу начать заниматься в спортзале, то с чего мне стоит начать? В первую очередь нужно оценить состояние своего здоровья и поставить цель. От этого будет зависеть, в каком направлении вы будете работать: силовые тренировки, пилатес, кроссфит, функциональный тренинг или что-то другое. Выберите удобную одежду, которая не сковывает движения, и кроссовки с амортизацией. Также стоит приобрести пульсометр, чтобы контролировать свой ритм и не перегружать сердце во время упражнений. Для каждого возраста существует свой максимальный диапазон частоты сердечных сокращений. Его можно рассчитать по формуле: (220- свой возраст) х 70%/80%/90%. Максимально допустимая частота для безопасного тренинга – 90%. ### Что мне делать в первый день в зале? При первом посещении вы должны записаться на вводный инструктаж, где тренер объяснит работу тренажеров и правильную технику выполнения упражнений. Он входит в стоимость абонемента. Как правило, в первый день вы только изуча

## Добавляем таблицу добавок

Для серьезных тренировок важны также пищевые добавки. Чтобы добавить информацию о них в нашего ассистента, мы нашли таблицу таких добавок в формате markdown:

In [76]:
with open("data/additives.md", encoding="utf-8") as f:
    additives = f.readlines()
additives_all = "".join(additives)

tokens = len(model.tokenize(additives_all))
print(f"Токенов: {tokens}, {len(additives_all)/tokens} chars/token")

Токенов: 6454, 5.105361016423923 chars/token


In [77]:
printx(additives_all[:1040])

| Название добавки                                 | Категория                                      | Назначение                                                                                                 | Доза                          | Рейтинг |
|--------------------------------------------------|-----------------------------------------------|-----------------------------------------------------------------------------------------------------------|-------------------------------|---------|
| Креатин                                          | Для наращивания мышечной массы, силы и ускорения восстановления | Увеличивает физическую силу, ускоряет восстановление; увеличивает массу                                   | 2-20 г в день                 | *****   |
| Глютамин                                         | Для наращивания мышечной массы, силы и ускорения восстановления | Предотвращает распад мышечной ткани; укрепляет иммунную систему                                           | 5-20 г в день                 | *****   

Видим, что табличка большая, поэтому её придётся *чанковать*. Но при этом важно чанковать табличку так, чтобы в каждом фрагмента оставался заголовок таблицы, который определяет семантику столбцов.

Отделим заголовок таблицы:

In [78]:
header = additives[:2]
header

['| Название добавки                                 | Категория                                      | Назначение                                                                                                 | Доза                          | Рейтинг |\n',
 '|--------------------------------------------------|-----------------------------------------------|-----------------------------------------------------------------------------------------------------------|-------------------------------|---------|\n']

Ниже будем чанковать табличку вручную, задав размер чанка в символах для простоты. Мы будем сразу загружать получившиеся фрагменты в облако, минуя диск:

In [79]:
chunk_size = 600 * 5  # около 600 tokens * 5 char/token

s = header.copy()
uploaded_additives = []
for x in additives[2:]:
    s.append(x)
    if len("".join(s)) > chunk_size:
        id = sdk.files.upload_bytes(
            "".join(s).encode(), ttl_days=5, expiration_policy="static",
            mime_type="text/markdown",
        )
        #printx("".join(s))
        uploaded_additives.append(id)
        s = header.copy()
print(f"Uploaded {len(uploaded_additives)} table chunks")

Uploaded 12 table chunks


Теперь добавим эти фрагменты в индекс:

In [80]:
op = index.add_files_deferred(uploaded_additives)
xfiles = op.wait()

Посмотрим, как система стала отвечать на вопросы о добавках:

In [81]:
thread = create_thread()

thread.write("Какие добавки помогают нарастить мышечную массу?")
result = assistant(thread)

printx(result.text)
print_citations(result)

Для наращивания мышечной массы могут помочь следующие добавки:

1. Креатин — увеличивает физическую силу, ускоряет восстановление и способствует увеличению мышечной массы.
2. Глютамин — предотвращает распад мышечной ткани и укрепляет иммунную систему.
3. Аминокислоты с разветвлёнными цепями — предотвращают распад мышечной ткани и притупляют чувство усталости.
4. ZMA (цинк-монометионин-аспартат, магний-аспартат, В6) — повышает уровень тестостерона и ИГФ-1, увеличивает физическую силу и улучшает сон/восстановление.
5. В-гидрокси-В-метилбутират (НМВ) — увеличивает мышечную массу тела и предотвращает распад мышечного белка.

------------------------
| Название добавки                                 | Категория                                      | Назначение                                                                                                 | Доза                          | Рейтинг |
|--------------------------------------------------|-----------------------------------------------|-----------------------------------------------------------------------------------------------------------|-------------------------------|---------|
| Глюкоманнан                                      | Для наращивания мышечной массы, силы и ускорения восстановления | Гелеобразующие волокна; пробиотик, укрепляющий иммунную систему; понижает уровень "плохого" холестерина; понижает уровень сахара в крови | 3-9 г в день                  | ****    |
| Экстракт гриба "шиитейк"                         | Для наращивания мышечной массы, силы и ускорения восстановления | Укрепляет иммунную систему                          

## Изменяем стратегию вызова поиска

В примере выше поисковый инструмент вызывался каждый раз в ответ на каждую реплику пользователя - этим он отличался от рассмотренного ранее механизма вызова инструментов, в котором LLM сама решала, когда сделать вызов функции. Постоянный вызов поиска может быть неэффективным, если большинство реплик отрабатываются с помощью знаний самой LLM.

Инструмент поиска в AI Assistant API позволяет указать стратегию вызова `function`, при которой поисковый инструмент будет вызываться также, как и другие инструменты - по решению LLM. Сделаем ассистента, который содержит в себе информацию о пищевых добавках, а также возможность записи сделанных упражнений.

> Со стратегие вызова поиска как инструмента нам будет сложно интегрировать информацию о фитнес-центре, поскольку сложно ясно сформулировать, когда именно нужно вызывать инструмент поиска. Для добавления общей информации в ассистента лучше использовать поисковый инструмент с постоянным вызовом.

Для начала создадим отдельный индекс для добавок:

In [82]:
op = sdk.search_indexes.create_deferred(
    uploaded_additives,
    index_type=HybridSearchIndexType(
        chunking_strategy=StaticIndexChunkingStrategy(
            max_chunk_size_tokens=1000, chunk_overlap_tokens=100
        ),
        combination_strategy=ReciprocalRankFusionIndexCombinationStrategy(),
    ),
)
additives_index = op.wait()

Теперь определим поисковый инструмент с функциональной стратегией вызова: 

In [112]:
search_tool_additives = sdk.tools.search_index(
    additives_index,
    call_strategy={
        'type': 'function',
        'function': {'name': 'additives', 'instruction': 'call this function whenever you need information on some food additives or medicines one can take to improve fitness results'}
    }
)

Наконец, создаём ассистента:

In [88]:
instruction = """
Ты - опытный фитнес-тренер, задача которого - помочь мне тренироваться в зале. Ты можешь
советовать упражнения, давать рекомендации по питанию и т.д. Ты можешь вести 
дневник выполненных пользователем упражнений - для этого используй функцию `Exercise`. Чтобы
показать список выполненных упражнений - используй `ListExercises`.
"""

assistant = Assistant([Exercise, ListExercises], instruction, search_tool=search_tool_additives)

thread = create_thread()
thread.write("Что нужно, чтобы начать заниматься в зале?")

res = assistant(thread)
print(f"Использовано документов: {len(res.citations)}")
printx(res.text)

Использовано документов: 0


Чтобы начать заниматься в зале, вам потребуется следующее:

1. **Медицинская консультация**: перед началом любых физических упражнений важно убедиться, что ваше здоровье позволяет вам заниматься спортом. Посетите врача и получите рекомендации по тренировкам.

2. **Спортивная одежда и обувь**: выберите удобную одежду, которая не будет сковывать движения, и специализированную обувь для зала, обеспечивающую хорошую амортизацию и поддержку стопы.

3. **План тренировок**: разработайте план тренировок с учетом ваших целей (похудение, набор мышечной массы, улучшение физической формы и т.д.) и уровня подготовки. Лучше всего обратиться к профессиональному тренеру, который поможет составить индивидуальный план.

4. **Базовые знания о технике выполнения упражнений**: правильная техника выполнения упражнений важна для предотвращения травм. Если вы новичок, рекомендуется начать с занятий с тренером, который научит вас правильной технике.

5. **Гидратация и питание**: обеспечьте достаточное потребление воды и сбалансированное питание, чтобы поддерживать энергию во время тренировок и восстановление после них.

6. **Мотивация и регулярность**: регулярные тренировки — ключ к успеху. Найдите источник мотивации, который будет помогать вам придерживаться графика занятий.

7. **Дневник тренировок**: ведите дневник, в котором будете фиксировать свои достижения, прогресс и ощущения от тренировок. Это поможет вам отслеживать свой путь и вносить необходимые коррективы в план тренировок.

In [89]:
thread.write("Какие добавки принимать для увеличения мышечной массы?")

res = assistant(thread)
print(f"Использовано документов: {len(res.citations)}")
printx(res.text)

 + Вызываю функцию additives, args = {'searchQuery': 'добавки для увеличения мышечной массы'}
Использовано документов: 5


Для увеличения мышечной массы часто используются следующие пищевые добавки:

1. **Креатин** — увеличивает физическую силу, ускоряет восстановление и способствует увеличению мышечной массы. Рекомендуемая доза: 2-20 г в день.
2. **Глютамин** — предотвращает распад мышечной ткани и укрепляет иммунную систему. Рекомендуемая доза: 5-20 г в день.
3. **Аминокислоты с разветвленными цепями (ВСАА)** — предотвращают распад мышечной ткани и притупляют чувство усталости. Рекомендуемая доза: 5-10 г в день.
4. **ZMA** — повышает уровень тестостерона и ИГФ-1, увеличивает физическую силу и улучшает сон/восстановление. Рекомендуемая доза: 2-3 капсулы в день.
5. **В-гидрокси-В-метилбутират (НМВ)** — увеличивает мышечную массу тела и предотвращает распад мышечного белка. Рекомендуемая доза: 3-5 г в день.

## Удаляем лишнее

В заключение удалим созданные ресурсы!

> **ВНИМАНИЕ**: Не выполняйте этот код, если у вас есть другие проекты с ассистентами в облаке! Он удаляет все индексы, ассистентов, файлы и переписки! 

In [47]:
for thread in sdk.threads.list():
    print(f" + deleting thread id={thread.id}",end="")
    try:
        thread.delete()
    except:
        print(" ! Error",end="")
    print()
        
for assistant in sdk.assistants.list():
    print(f" + deleting assistant id={assistant.id}")
    assistant.delete()

 + deleting thread id=fvtiinvhmmkgke8mjd7h
 + deleting thread id=fvtun3vd77bme790696h
 + deleting thread id=fvtl8v29c3308dgeu5re
 + deleting thread id=fvtv1s8ilejs0j8u2vkf
 + deleting thread id=fvt78dit8d80e09iidfk
 + deleting thread id=fvt5dbdnnpebsgasimer
 + deleting thread id=fvt3a7b4v5bju9dehp8k
 + deleting thread id=fvt6sj220if7o6ntas5q
 + deleting thread id=fvtice1ne174h5u0vd66
 + deleting thread id=fvt9utcfh382b3n2j4pp
 + deleting thread id=fvtfp9gaf0bv0omgdbgo
 + deleting thread id=fvt8b6nlusdt1j2bhi2o
 + deleting thread id=fvtq733vf8utvqett9t6
 + deleting thread id=fvtl7ce2bnvcsgajdtrf
 + deleting thread id=fvtbd4k1csg2hviq0l6q
 + deleting thread id=fvt0jq1hkfoki6sr6s62
 + deleting thread id=fvtb7unriq6944koonus
 + deleting thread id=fvt9a74fsprjq29boc5f
 + deleting assistant id=fvt6cipttbtqkckn3hoa
 + deleting assistant id=fvt1j5rkg30mgaj9ioog
 + deleting assistant id=fvtjfeot10qvbai02orh
 + deleting assistant id=fvtjtri887d3mdcsvsla
 + deleting assistant id=fvtbk3rheoktemftr

In [49]:
from tqdm.auto import tqdm

for index in sdk.search_indexes.list():
    print(f" + deleting index id={index.id}")
    index.delete()
    
print(" + Deleting files")
for file in tqdm(sdk.files.list()):
    # print(f" + deleting file id={file.id}")
    file.delete()

 + Deleting files


0it [00:00, ?it/s]

0it [00:01, ?it/s]
